In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor, LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from catboost import CatBoostRegressor
import os
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl
import kaggle_evaluation.mcts_inference_server

In [ ]:
# Paths
train_path = "/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"



In [ ]:
# Load data
train_data = pd.read_csv(train_path)

In [ ]:
train_data

In [ ]:

# Drop columns with 95% or more identical values and columns with all null values
def drop_column_name(df):
    drop_columns = []
    for column in df.columns:
        if df[column].value_counts().max() / len(df) >= 0.95 or df[column].isnull().all():
            drop_columns.append(column)
    return drop_columns

drop_column = drop_column_name(train_data)
drop_column.extend(["Id", "EnglishRules", "LudRules", "num_wins_agent1", "num_draws_agent1", "num_losses_agent1"])

drop_column.extend(['GameRulesetName', 'agent1', 'agent2'])

In [ ]:
# Prepare train dataset
train = train_data.drop(columns=drop_column, errors='ignore')
# le1 = LabelEncoder()
# le2 = LabelEncoder()
# le3 = LabelEncoder()
# train["GameRulesetName"] = le1.fit_transform(train["GameRulesetName"])
# train["agent1"] = le2.fit_transform(train["agent1"])
# train["agent2"] = le3.fit_transform(train["agent2"])

# Split data
X = train.drop('utility_agent1', axis=1)  # Features
y = train['utility_agent1']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)





In [ ]:
# Define model training function
def train_model(X_train, y_train):
    model = XGBRegressor(n_estimators=150, learning_rate=0.05, max_depth=6)
    model.fit(X_train, y_train)
    print(222)
    return model


def clean_data(test_df):
    """
    Cleans the test dataframe by converting all applicable columns to numeric and handling errors.
    """
    # Drop columns that are not required
    test_df = test_df.drop(columns=drop_column, errors='ignore')
    
    # Ensure the columns are present
#     required_columns = ['GameRulesetName', 'agent1', 'agent2']
#     for col in required_columns:
#         if col not in test_df.columns:
#             test_df[col] = -1  # Assign a default value or handle missing columns appropriately

    # Transform categorical data using pre-fitted LabelEncoders
#     try:
#         test_df["GameRulesetName"] = le1.transform(test_df["GameRulesetName"])
#     except KeyError:
#         test_df["GameRulesetName"] = -1  # Assign a default value for unseen categories
    
#     try:
#         test_df["agent1"] = le2.transform(test_df["agent1"])
#     except KeyError:
#         test_df["agent1"] = -1
    
#     try:
#         test_df["agent2"] = le3.transform(test_df["agent2"])
#     except KeyError:
#         test_df["agent2"] = -1

    # Fill missing values
    test_df = test_df.fillna(0)
    print(333)
    return test_df

In [ ]:
counter = 0
model = None  # Initialize model as None globally

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter, model, X_train, y_train  # Declare all global variables at the start
    print(1)
    test_df = test.to_pandas()

    # Clean the data
    test_df = clean_data(test_df)
    print(2)

    # Train the model only once
    if counter == 0:
        model = train_model(X_train, y_train)
        counter += 1

    print(3)

    # Return NaN values on failure
    try:
        predictions = model.predict(test_df)
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None

    # Summation step: add a constant value to the predictions
    
    print(predictions)
    print(type(predictions))

    # Prepare the final submission format
    submission = sample_sub.with_columns(pl.Series("utility_agent1", predictions))
    print(submission)
    print(type(submission))

    return submission


In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )    
    )